In [1]:
%matplotlib widget
import numpy as np
import matplotlib.pyplot as plt 
import pandas as pd 

In [6]:
solar_data_filename = '/Users/brad/Desktop/CS6620/Data/Sample.csv' 
weather_data_filename = '/Users/brad/Desktop/CS6620/Data/Weather.csv' 

In [15]:
solarData = pd.read_csv(solar_data_filename, sep=',')

In [16]:
solarData.head()

,Unnamed: 0,TIME,Solar_power,Meter_value
0,0,2019-07-11 13:11:55,-5.366741e+08,5.342115
1,1,2019-07-11 13:13:30,-5.365808e+08,5.352649
2,2,2019-07-11 13:13:38,-5.368667e+08,5.352649
3,3,2019-07-11 13:13:45,-5.369603e+08,5.352649
4,4,2019-07-11 13:13:53,-5.371526e+08,5.352649


In [18]:
solarData.drop('Meter_value')
solarData.head()

,Unnamed: 0,TIME,Solar_power,Meter_value
1375821,1375821,2021-03-09 10:40:24,4319.288815,4.963623
1375822,1375822,2021-03-09 10:41:08,4281.411649,3.418124
1375823,1375823,2021-03-09 10:41:08,4290.056625,3.418124
1375824,1375824,2021-03-09 10:41:52,3726.992754,2.582676
1375825,1375825,2021-03-09 10:41:52,3635.134951,2.582676


In [20]:
solarData = solarData.drop(['Meter_value'], axis=1)
solarData.head()

,Unnamed: 0,TIME,Solar_power
0,0,2019-07-11 13:11:55,-5.366741e+08
1,1,2019-07-11 13:13:30,-5.365808e+08
2,2,2019-07-11 13:13:38,-5.368667e+08
3,3,2019-07-11 13:13:45,-5.369603e+08
4,4,2019-07-11 13:13:53,-5.371526e+08


In [21]:
solarData.tail(100)

,Unnamed: 0,TIME,Solar_power
1375726,1375726,2021-03-09 10:05:34,3716.617282
1375727,1375727,2021-03-09 10:05:35,3632.959263
1375728,1375728,2021-03-09 10:06:19,3608.891261
1375729,1375729,2021-03-09 10:06:19,3628.735385
1375730,1375730,2021-03-09 10:07:03,3791.987588
...,...,...,...
1375821,1375821,2021-03-09 10:40:24,4319.288815
1375822,1375822,2021-03-09 10:41:08,4281.411649
1375823,1375823,2021-03-09 10:41:08,4290.056625
1375824,1375824,2021-03-09 10:41:52,3726.992754


In [37]:
time = solarData['TIME'].tail(160)
print(time.head())
print(time.tail())


1375666    2021-03-09 09:43:20
1375667    2021-03-09 09:43:20
1375668    2021-03-09 09:44:04
1375669    2021-03-09 09:44:04
1375670    2021-03-09 09:44:49
Name: TIME, dtype: object
1375821    2021-03-09 10:40:24
1375822    2021-03-09 10:41:08
1375823    2021-03-09 10:41:08
1375824    2021-03-09 10:41:52
1375825    2021-03-09 10:41:52
Name: TIME, dtype: object


In [40]:
plt.figure()
plt.plot(solarData['TIME'].tail(160), solarData['Solar_power'].tail(160))
plt.xlabel("Time")
plt.ylabel("Solar Generation")
plt.title("ASPIRE Labs data 3-9-2021 (9:40 - 10:40 am)")
plt.xticks([])
plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [41]:
weatherpf = pd.read_csv(weather_data_filename, sep=',')
weatherpf.tail()

,date_time,station_id,rh,eto,etr,rso,ea_avg,precip,rh_max,rh_min,...,winds_max,winds_tmx,netrad_avg,fan_rmp_avg,visibilitymiles,batt_volt_min,surfacet1_avg,surfacet2_avg,visibilitymiles_avg,pressurekpasealevel
14286,2021-03-07 19:59:59,1279257,43,0.0,0.000,0.0,0.463,0.0,45,40,...,9.171,07MAR2021:19:20:57,-46.20,4293,17.274,12.27,37.0,-42.7,17.958,101.2
14287,2021-03-07 20:59:59,1279257,45,0.0,0.000,0.0,0.475,0.0,50,41,...,8.948,07MAR2021:20:53:29,-42.18,4286,17.958,12.27,32.4,-55.1,17.274,101.2
14288,2021-03-07 21:59:59,1279257,41,0.0,-0.001,0.0,0.430,0.0,43,38,...,8.053,07MAR2021:21:29:30,-44.46,4274,18.455,12.24,35.1,-46.1,18.330,101.1
14289,2021-03-07 22:59:59,1279257,39,0.0,0.000,0.0,0.409,0.0,42,37,...,12.527,07MAR2021:22:12:59,-55.05,4266,18.268,12.23,35.6,-40.9,18.330,101.0
14290,2021-03-07 23:59:59,1279257,44,0.0,0.000,0.0,0.407,0.0,50,38,...,14.093,07MAR2021:23:27:36,-52.82,4259,18.641,12.16,37.4,-36.2,18.579,100.9
